# Understanding Resnet Model Features

We know that the Resnet model works well, but why does it work? How can we have confidence that it is searching out the correct features? A recent paper, [Axiomatic Attribution for Deep Networks](https://arxiv.org/pdf/1703.01365.pdf), shows that averaging gradients taken along a path of images from a blank image (e.g. pure black or grey) to the actual image, can robustly predict sets of pixels that have a strong impact on the overall classification of the image. The below code shows how to modify the TF estimator code to analyze model behavior of different images.

In [0]:
import csv
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import urllib
import tensorflow as tf

from subprocess import call

# Constants

In [0]:
_DEFAULT_IMAGE_SIZE = 224
_NUM_CHANNELS = 3
_LABEL_CLASSES = 1001

_MOMENTUM = 0.9
_WEIGHT_DECAY = 1e-4

_BATCH_NORM_DECAY = 0.997
_BATCH_NORM_EPSILON = 1e-5

RESNET_SIZE = 50  # We're loading a resnet-50 saved model.

# Model directory
MODEL_DIR='resnet_model_checkpoints'
VIS_DIR='visualization'

# RIEMANN STEPS is the number of steps in a Riemann Sum.
# This is used to compute an approximate the integral of gradients by supplying
# images on the path from a blank image to the original image.
RIEMANN_STEPS = 50

# Return the top k classes and probabilities, so we can also visualize model inference
# against other contending classes besides the most likely class.
TOP_K = 5


# Download model checkpoint

The next step is to load the researcher's saved checkpoint into our estimator. We will download it from
http://download.tensorflow.org/models/official/resnet50_2017_11_30.tar.gz using the following commands.

In [0]:
import urllib.request

urllib.request.urlretrieve("http://download.tensorflow.org/models/official/resnet50_2017_11_30.tar.gz ", "resnet.tar.gz")

In [0]:
#unzip the file into a directory called resnet
call(["mkdir", MODEL_DIR])
call(["tar", "-zxvf", "resnet.tar.gz", "-C", MODEL_DIR])

In [0]:
# Make sure you see model checkpoint files in this directory
os.listdir(MODEL_DIR)

# Helper functions
 
 These helper functions define the neural network that was used to train the model. When we load a trained estimator checkpoint, we need to ensure that the neural network definitions are IDENTICAL to that used in training, since the checkpoint will include weights and bias values pertaining to specific neurons in the network.

In [0]:
def batch_norm_relu(inputs, is_training, data_format):
  """Performs a batch normalization followed by a ReLU."""
  # We set fused=True for a significant performance boost. See
  # https://www.tensorflow.org/performance/performance_guide#common_fused_ops
  inputs = tf.layers.batch_normalization(
      inputs=inputs, axis=1 if data_format == 'channels_first' else 3,
      momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON, center=True,
      scale=True, training=is_training, fused=True)
  inputs = tf.nn.relu(inputs)
  return inputs


def fixed_padding(inputs, kernel_size, data_format):
  """Pads the input along the spatial dimensions independently of input size.
  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    kernel_size: The kernel to be used in the conv2d or max_pool2d operation.
                 Should be a positive integer.
    data_format: The input format ('channels_last' or 'channels_first').
  Returns:
    A tensor with the same format as the input with the data either intact
    (if kernel_size == 1) or padded (if kernel_size > 1).
  """
  pad_total = kernel_size - 1
  pad_beg = pad_total // 2
  pad_end = pad_total - pad_beg

  if data_format == 'channels_first':
    padded_inputs = tf.pad(inputs, [[0, 0], [0, 0],
                                    [pad_beg, pad_end], [pad_beg, pad_end]])
  else:
    padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg, pad_end],
                                    [pad_beg, pad_end], [0, 0]])
  return padded_inputs


def conv2d_fixed_padding(inputs, filters, kernel_size, strides, data_format):
  """Strided 2-D convolution with explicit padding."""
  # The padding is consistent and is based only on `kernel_size`, not on the
  # dimensions of `inputs` (as opposed to using `tf.layers.conv2d` alone).
  if strides > 1:
    inputs = fixed_padding(inputs, kernel_size, data_format)

  return tf.layers.conv2d(
      inputs=inputs, filters=filters, kernel_size=kernel_size, strides=strides,
      padding=('SAME' if strides == 1 else 'VALID'), use_bias=False,
      kernel_initializer=tf.variance_scaling_initializer(),
      data_format=data_format)


def building_block(inputs, filters, is_training, projection_shortcut, strides,
                   data_format):
  """Standard building block for residual networks with BN before convolutions.
  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    filters: The number of filters for the convolutions.
    is_training: A Boolean for whether the model is in training or inference
      mode. Needed for batch normalization.
    projection_shortcut: The function to use for projection shortcuts (typically
      a 1x1 convolution when downsampling the input).
    strides: The block's stride. If greater than 1, this block will ultimately
      downsample the input.
    data_format: The input format ('channels_last' or 'channels_first').
  Returns:
    The output tensor of the block.
  """
  shortcut = inputs
  inputs = batch_norm_relu(inputs, is_training, data_format)

  # The projection shortcut should come after the first batch norm and ReLU
  # since it performs a 1x1 convolution.
  if projection_shortcut is not None:
    shortcut = projection_shortcut(inputs)

  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=3, strides=strides,
      data_format=data_format)

  inputs = batch_norm_relu(inputs, is_training, data_format)
  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=3, strides=1,
      data_format=data_format)

  return inputs + shortcut


def bottleneck_block(inputs, filters, is_training, projection_shortcut,
                     strides, data_format):
  """Bottleneck block variant for residual networks with BN before convolutions.
  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    filters: The number of filters for the first two convolutions. Note that the
      third and final convolution will use 4 times as many filters.
    is_training: A Boolean for whether the model is in training or inference
      mode. Needed for batch normalization.
    projection_shortcut: The function to use for projection shortcuts (typically
      a 1x1 convolution when downsampling the input).
    strides: The block's stride. If greater than 1, this block will ultimately
      downsample the input.
    data_format: The input format ('channels_last' or 'channels_first').
  Returns:
    The output tensor of the block.
  """
  shortcut = inputs
  inputs = batch_norm_relu(inputs, is_training, data_format)

  # The projection shortcut should come after the first batch norm and ReLU
  # since it performs a 1x1 convolution.
  if projection_shortcut is not None:
    shortcut = projection_shortcut(inputs)

  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=1, strides=1,
      data_format=data_format)

  inputs = batch_norm_relu(inputs, is_training, data_format)
  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=3, strides=strides,
      data_format=data_format)

  inputs = batch_norm_relu(inputs, is_training, data_format)
  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=4 * filters, kernel_size=1, strides=1,
      data_format=data_format)

  return inputs + shortcut


def block_layer(inputs, filters, block_fn, blocks, strides, is_training, name,
                data_format):
  """Creates one layer of blocks for the ResNet model.
  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    filters: The number of filters for the first convolution of the layer.
    block_fn: The block to use within the model, either `building_block` or
      `bottleneck_block`.
    blocks: The number of blocks contained in the layer.
    strides: The stride to use for the first convolution of the layer. If
      greater than 1, this layer will ultimately downsample the input.
    is_training: Either True or False, whether we are currently training the
      model. Needed for batch norm.
    name: A string name for the tensor output of the block layer.
    data_format: The input format ('channels_last' or 'channels_first').
  Returns:
    The output tensor of the block layer.
  """
  # Bottleneck blocks end with 4x the number of filters as they start with
  filters_out = 4 * filters if block_fn is bottleneck_block else filters

  def projection_shortcut(inputs):
    return conv2d_fixed_padding(
        inputs=inputs, filters=filters_out, kernel_size=1, strides=strides,
        data_format=data_format)

  # Only the first block per block_layer uses projection_shortcut and strides
  inputs = block_fn(inputs, filters, is_training, projection_shortcut, strides,
                    data_format)

  for _ in range(1, blocks):
    inputs = block_fn(inputs, filters, is_training, None, 1, data_format)

  return tf.identity(inputs, name)


def imagenet_resnet_v2_generator(block_fn, layers, num_classes,
                                 data_format=None):
  """Generator for ImageNet ResNet v2 models.
  Args:
    block_fn: The block to use within the model, either `building_block` or
      `bottleneck_block`.
    layers: A length-4 array denoting the number of blocks to include in each
      layer. Each layer consists of blocks that take inputs of the same size.
    num_classes: The number of possible classes for image classification.
    data_format: The input format ('channels_last', 'channels_first', or None).
      If set to None, the format is dependent on whether a GPU is available.
  Returns:
    The model function that takes in `inputs` and `is_training` and
    returns the output tensor of the ResNet model.
  """
  if data_format is None:
    data_format = (
        'channels_first' if tf.test.is_built_with_cuda() else 'channels_last')

  def model(inputs, is_training):
    """Constructs the ResNet model given the inputs."""
    if data_format == 'channels_first':
      # Convert the inputs from channels_last (NHWC) to channels_first (NCHW).
      # This provides a large performance boost on GPU. See
      # https://www.tensorflow.org/performance/performance_guide#data_formats
      inputs = tf.transpose(inputs, [0, 3, 1, 2])

    inputs = conv2d_fixed_padding(
        inputs=inputs, filters=64, kernel_size=7, strides=2,
        data_format=data_format)
    inputs = tf.identity(inputs, 'initial_conv')
    inputs = tf.layers.max_pooling2d(
        inputs=inputs, pool_size=3, strides=2, padding='SAME',
        data_format=data_format)
    inputs = tf.identity(inputs, 'initial_max_pool')

    inputs = block_layer(
        inputs=inputs, filters=64, block_fn=block_fn, blocks=layers[0],
        strides=1, is_training=is_training, name='block_layer1',
        data_format=data_format)
    inputs = block_layer(
        inputs=inputs, filters=128, block_fn=block_fn, blocks=layers[1],
        strides=2, is_training=is_training, name='block_layer2',
        data_format=data_format)
    inputs = block_layer(
        inputs=inputs, filters=256, block_fn=block_fn, blocks=layers[2],
        strides=2, is_training=is_training, name='block_layer3',
        data_format=data_format)
    inputs = block_layer(
        inputs=inputs, filters=512, block_fn=block_fn, blocks=layers[3],
        strides=2, is_training=is_training, name='block_layer4',
        data_format=data_format)

    inputs = batch_norm_relu(inputs, is_training, data_format)
    inputs = tf.layers.average_pooling2d(
        inputs=inputs, pool_size=7, strides=1, padding='VALID',
        data_format=data_format)
    inputs = tf.identity(inputs, 'final_avg_pool')
    inputs = tf.reshape(inputs,
                        [-1, 512 if block_fn is building_block else 2048])
    inputs = tf.layers.dense(inputs=inputs, units=num_classes)
    inputs = tf.identity(inputs, 'final_dense')
    return inputs

  return model


def imagenet_resnet_v2(resnet_size, num_classes, data_format=None):
  """Returns the ResNet model for a given size and number of output classes."""
  model_params = {
      18: {'block': building_block, 'layers': [2, 2, 2, 2]},
      34: {'block': building_block, 'layers': [3, 4, 6, 3]},
      50: {'block': bottleneck_block, 'layers': [3, 4, 6, 3]},
      101: {'block': bottleneck_block, 'layers': [3, 4, 23, 3]},
      152: {'block': bottleneck_block, 'layers': [3, 8, 36, 3]},
      200: {'block': bottleneck_block, 'layers': [3, 24, 36, 3]}
  }

  if resnet_size not in model_params:
    raise ValueError('Not a valid resnet_size:', resnet_size)

  params = model_params[resnet_size]
  return imagenet_resnet_v2_generator(
      params['block'], params['layers'], num_classes, data_format)

# Image preprocessing functions

Note that preprocessing functions are called during training as well (see https://github.com/tensorflow/models/blob/master/official/resnet/imagenet_main.py and https://github.com/tensorflow/models/blob/master/official/resnet/vgg_preprocessing.py), so we will need to extract relevant logic from these functions. Below is a simplified preprocessing code that normalizes the image's pixel values.

For simplicity, we assume the client provides properly-sized images 224 x 224 x 3 in batches. It will become clear later that sending images over ip in protobuf format can be more easily handled by storing a 4d tensor. The only preprocessing required here is to subtract the mean.

In [0]:
def preprocess_images(images):
  """Preprocesses the image by subtracting out the mean from all channels.
  Args:
    image: A 4D `Tensor` representing a batch of images.
  Returns:
    image pixels normalized to be between -0.5 and 0.5
  """
  return tf.to_float(images) / 255 - 0.5

# Resnet Model Functions

We are going to create two estimators here since we need to run two model predictions. 

* The first prediction computes the top labels for the image by returning the argmax_k top logits. 

* The second prediction returns a sequence of gradients along the straightline path from a purely grey image (127.5, 127.5, 127.5) to the final image. We use grey here because the resnet model transforms this pixel value to all 0s.

Below is the resnet model function.

In [0]:
def resnet_model_fn(features, labels, mode):
  """Our model_fn for ResNet to be used with our Estimator."""

  # Preprocess images as necessary for resnet
  features = preprocess_images(features['images'])

  # This network must be IDENTICAL to that used to train.
  network = imagenet_resnet_v2(RESNET_SIZE, _LABEL_CLASSES)

  # tf.estimator.ModeKeys.TRAIN will be false since we are predicting.
  logits = network(
      inputs=features, is_training=(mode == tf.estimator.ModeKeys.TRAIN))

  # Instead of the top 1 result, we can now return top k!
  top_k_logits, top_k_classes = tf.nn.top_k(logits, k=TOP_K)
  top_k_probs = tf.nn.softmax(top_k_logits)
  predictions = {
      'classes': top_k_classes,
      'probabilities': top_k_probs
  }


  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=predictions, 
  )


# Gradients Model Function

The Gradients model function takes as input a single image (a 4d tensor of dimension [1, 244, 244, 3]) and expands it to a series of images (tensor dimension [RIEMANN_STEPS + 1, 244, 244, 3]), where each image is simply a "fractional" image, with image 0 being pure gray to image RIEMANN_STEPS being the original image. The gradients are then computed for each of these images, and various outputs are returned.

**Note:** Each step is a single inference that returns an entire gradient pixel map.
The total gradient map evaluation can take a couple minutes!


In [0]:
def gradients_model_fn(features, labels, mode):
  """Our model_fn for ResNet to be used with our Estimator."""
    
  # Supply labels from features dict since prediction mode causes labels to be none
  labels = features['labels']
    
  # Features here is a 4d tensor of ONE image. Normalize it as in training and serving.
  features = preprocess_images(features['images'])

  # This network must be IDENTICAL to that used to train.
  network = imagenet_resnet_v2(RESNET_SIZE, _LABEL_CLASSES)

  # path_features should have dim [RIEMANN_STEPS + 1, 224, 224, 3]
  path_features = tf.zeros([1, 224, 224, 3])
  for i in range(1, RIEMANN_STEPS + 1):
    path_features = tf.concat([path_features, features * i / RIEMANN_STEPS], axis=0)
   
  # Path logits should evaluate logits for each path feature and return a 2d array for all path images and classes
  path_logits = network(inputs=path_features, is_training=(mode == tf.estimator.ModeKeys.TRAIN))

  # The logit we care about is only that pertaining to the label. Labels has only one element, so retrieve it.
  target_logits = path_logits[:, labels[0]]
   
  # Compute gradients for each image with respect to each logit
  gradients = tf.gradients(target_logits, path_features)
    
  # Multiply elementwise to the original image to get weighted gradients for each pixel.
  gradients = tf.squeeze(tf.multiply(gradients, features))
    
  predictions = {
      'path_features': path_features,  # for debugging
      'path_logits': path_logits,  # for debugging
      'target_logits': target_logits,  # use this to verify that the riemann integral works out
      'path_features': path_features, # for displaying path images
      'gradients': gradients  # for displaying gradient images and computing integrated gradient
  }


  return tf.estimator.EstimatorSpec(
    mode=mode,
    predictions=predictions,  # This is the returned value
  )


# Estimators

Load in the model_fn using the checkpoints from MODEL_DIR. This will initialize our weights which we will then use to run backpropagation to find integrated gradients.

In [0]:
# Load this model into our estimator
resnet_estimator = tf.estimator.Estimator(
  model_fn=resnet_model_fn,  # Call our generate_model_fn to create model function
  model_dir=MODEL_DIR,  # Where to look for model checkpoints
  #config not needed
)

gradients_estimator = tf.estimator.Estimator(
  model_fn=gradients_model_fn,  # Call our generate_model_fn to create model function
  model_dir=MODEL_DIR,  # Where to look for model checkpoints
  #config not needed
)

# Create properly sized image in numpy

Load whatever image you would like (local or url), and resize to 224 x 224 x 3 using opencv2.

In [0]:
def resize_and_pad_image(img, output_image_dim=_DEFAULT_IMAGE_SIZE):
  """Resize the image to make it IMAGE_DIM x IMAGE_DIM pixels in size.

  If an image is not square, it will pad the top/bottom or left/right
  with black pixels to ensure the image is square.

  Args:
    img: the input 3-color image
    output_image_dim: resized and padded output length (and width)

  Returns:
    resized and padded image
  """

  h, w = img.shape[:2]

  # interpolation method
  if h > output_image_dim or w > output_image_dim:
    # use preferred interpolation method for shrinking image
    interp = cv2.INTER_AREA
  else:
    # use preferred interpolation method for stretching image
    interp = cv2.INTER_CUBIC

  # aspect ratio of image
  aspect = float(w) / h

  # compute scaling and pad sizing
  if aspect > 1:  # Image is "wide". Add black pixels on top and bottom.
    new_w = output_image_dim
    new_h = np.round(new_w / aspect)
    pad_vert = (output_image_dim - new_h) / 2
    pad_top, pad_bot = int(np.floor(pad_vert)), int(np.ceil(pad_vert))
    pad_left, pad_right = 0, 0
  elif aspect < 1:  # Image is "tall". Add black pixels on left and right.
    new_h = output_image_dim
    new_w = np.round(new_h * aspect)
    pad_horz = (output_image_dim - new_w) / 2
    pad_left, pad_right = int(np.floor(pad_horz)), int(np.ceil(pad_horz))
    pad_top, pad_bot = 0, 0
  else:  # square image
    new_h = output_image_dim
    new_w = output_image_dim
    pad_left, pad_right, pad_top, pad_bot = 0, 0, 0, 0

  # scale to IMAGE_DIM x IMAGE_DIM and pad with zeros (black pixels)
  scaled_img = cv2.resize(img, (int(new_w), int(new_h)), interpolation=interp)
  scaled_img = cv2.copyMakeBorder(scaled_img,
                                  pad_top, pad_bot, pad_left, pad_right,
                                  borderType=cv2.BORDER_CONSTANT, value=[127, 127, 127]) # Grey border
  return scaled_img

In [0]:
IMAGE = 'http://homecookingadventure.com/images/recipes/Chocolate_Mirror_Cake_main.jpg'
IMAGE_NAME = os.path.splitext(os.path.basename(IMAGE))[0]
print(IMAGE_NAME)

In [0]:
feature = None
if 'http' in IMAGE:
  resp = urllib.request.urlopen(IMAGE)
  feature = np.asarray(bytearray(resp.read()), dtype='uint8')
  feature = cv2.imdecode(feature, cv2.IMREAD_COLOR)
else:
  feature = cv2.imread(IMAGE)  # Parse the image from your local disk.

feature = cv2.cvtColor(feature, cv2.COLOR_BGR2RGB)  # Flip the RGB (cv2 issue)
# Resize and pad the image
feature = resize_and_pad_image(feature)
# Append to features_array
feature = np.array([feature], dtype='uint8')

In [0]:
# Display the image to validate
imgplot = plt.imshow(feature[0])
plt.show()

# Prediction Input Function

Since we are analyzing the model using the estimator api, we need to provide an input function for prediction. Fortunately, there are built-in input functions that can read from numpy arrays, e.g. tf.estimator.inputs.numpy_input_fn.

In [0]:
label_predictions = resnet_estimator.predict(
    tf.estimator.inputs.numpy_input_fn(
        x={'images': feature},
        shuffle=False
    )
)

label_dict = next(label_predictions)


In [0]:
# Print out probabilities and class names
classval = label_dict['classes']
probsval = label_dict['probabilities']
labels = []
with open('imagenet1000_clsid_to_human.txt', 'r') as f:
  label_reader = csv.reader(f, delimiter=':', quotechar='\'')
  for row in label_reader:
    labels.append(row[1][:-1])
# The served model uses 0 as the miscellaneous class, and so starts indexing
# the imagenet images from 1. Subtract 1 to reference the text correctly.
classval = [labels[x - 1] for x in classval]
class_and_probs = [str(p) + ' : ' + c for c, p in zip(classval, probsval)]
for j in range(0, 5):
  print(class_and_probs[j])

# Computing Gradients

Run the gradients estimator to retrieve a generator of metrics and gradient pictures, and pickle the images. 

In [0]:
# make the visualization directory
IMAGE_DIR = os.path.join(VIS_DIR, IMAGE_NAME)
call(['mkdir', '-p', IMAGE_DIR])


In [0]:
# Get one of the top classes. 0 picks out the best, 1 picks out second best, etc...
best_label = label_dict['classes'][0]

# Compute gradients with respect to this class
gradient_predictions = gradients_estimator.predict(
    tf.estimator.inputs.numpy_input_fn(
        x={'images': feature, 'labels': np.array([best_label])},
        shuffle=False
    )
)

# Start computing the sum of gradients (to be used for integrated gradients)
int_gradients = np.zeros((224, 224, 3))
gradients_and_logits = []

# Print gradients along the path, and pickle them
for i in range(0, RIEMANN_STEPS + 1):
    gradient_dict = next(gradient_predictions)
    gradient_map = gradient_dict['gradients']
    print('Path image %d: gradient: %f, logit: %f' % (i, np.sum(gradient_map), gradient_dict['target_logits']))
    # Gradient visualization output pickles
    pickle.dump(gradient_map, open(os.path.join(IMAGE_DIR, 'path_gradient_' + str(i) + '.pkl'), "wb" ))
    int_gradients = np.add(int_gradients, gradient_map)
    gradients_and_logits.append((np.sum(gradient_map), gradient_dict['target_logits']))
    
pickle.dump(int_gradients, open(os.path.join(IMAGE_DIR, 'int_gradients.pkl'), "wb" ))
pickle.dump(gradients_and_logits, open(os.path.join(IMAGE_DIR, 'gradients_and_logits.pkl'), "wb" ))

# Visualization

If you simply want to play around with visualization, unpickle the result from above so you do not have to rerun prediction again. The following visualizes the gradients with different amplification of pixels, and prints their derivatives and logits as well to view where the biggest differentiators lie. You can also modify the INTERPOLATION flag to increase the "fatness" of pixels.

Below are two examples of visualization methods: one computing the gradient value normalized to between 0 and 1, and another visualizing absolute deviation from the median.

## Plotting individual image gradients along path

First, let us plot the individual gradient value for all gradient path images. Pay special attention to the images with a large positive gradient (i.e. in the direction of increasing logit for the most likely class). Do the pixel gradients resemble the image class you are trying to detect?

In [0]:
AMPLIFICATION = 2.0
INTERPOLATION = 'none'

gradients_and_logits = pickle.load(open(os.path.join(IMAGE_DIR, 'gradients_and_logits.pkl'), "rb" ))
for i in range(0, RIEMANN_STEPS + 1):
    gradient_map = pickle.load(open(os.path.join(IMAGE_DIR, 'path_gradient_' + str(i) + '.pkl'), "rb" ))
    min_grad = np.ndarray.min(gradient_map)
    max_grad = np.ndarray.max(gradient_map)
    median_grad = np.median(gradient_map)
    gradient_and_logit = gradients_and_logits[i]

    plt.figure(figsize=(10,10))
    plt.subplot(121)
    plt.title('Image %d: grad: %.2f, logit: %.2f' % (i, gradient_and_logit[0], gradient_and_logit[1]))
    imgplot = plt.imshow((gradient_map - min_grad) / (max_grad - min_grad),
                        interpolation=INTERPOLATION)
    plt.subplot(122)
    plt.title('Image %d: grad: %.2f, logit: %.2f' % (i, gradient_and_logit[0], gradient_and_logit[1]))
    imgplot = plt.imshow(np.abs(gradient_map - median_grad) * AMPLIFICATION / max(max_grad - median_grad, median_grad - min_grad),
                       interpolation=INTERPOLATION)
    plt.show()

## Plot the Integrated Gradient

When integrating over all gradients along the path, the result is an image that captures larger signals from pixels with the large gradients. Is the integrated gradient a clear representation of what it is trying to detect?

In [0]:
AMPLIFICATION = 2.0
INTERPOLATION = 'none'

# Plot the integrated gradients
int_gradients = pickle.load(open(os.path.join(IMAGE_DIR, 'int_gradients.pkl'), "rb" ))
min_grad = np.ndarray.min(int_gradients)
max_grad = np.ndarray.max(int_gradients)
median_grad = np.median(int_gradients)
plt.figure(figsize=(15,15))
plt.subplot(131)
imgplot = plt.imshow((int_gradients - min_grad) / (max_grad - min_grad),
                    interpolation=INTERPOLATION)
plt.subplot(132)
imgplot = plt.imshow(np.abs(int_gradients - median_grad) * AMPLIFICATION / max(max_grad - median_grad, median_grad - min_grad),
                        interpolation=INTERPOLATION)
plt.subplot(133)
imgplot = plt.imshow(feature[0])
plt.show()

# Verify that the average of gradients is equal to the difference in logits
print('total logit diff: %f' % (gradients_and_logits[RIEMANN_STEPS][1] - gradients_and_logits[0][1]))
print('sum of integrated gradients: %f' % (np.sum(int_gradients) / RIEMANN_STEPS + 1))

## Plot the integrated gradients for each channel

We can also visualize individual pixel contributions from different RGB channels.

Can you think of any other visualization ideas to try out?

In [0]:
AMPLIFICATION = 2.0
INTERPOLATION = 'none'

# Show red-green-blue channels for integrated gradients
for channel in range(0, 3):
    gradient_channel = int_gradients[:,:,channel]
    min_grad = np.ndarray.min(gradient_channel)
    max_grad = np.ndarray.max(gradient_channel)
    median_grad = np.median(gradient_channel)
    plt.figure(figsize=(10,10))
    plt.subplot(121)
    imgplot = plt.imshow((gradient_channel - min_grad) / (max_grad - min_grad),
                         interpolation=INTERPOLATION,
                         cmap='gray')
    plt.subplot(122)
    imgplot = plt.imshow(np.abs(gradient_channel - median_grad) * AMPLIFICATION / max(max_grad - median_grad, median_grad - min_grad),
                         interpolation=INTERPOLATION,
                         cmap='gray')
    plt.show()